In [2]:
import os
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import pickle
from keras.optimizers import RMSprop
import numpy as np
from numpy import array

2023-02-16 19:34:58.198841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
documents = []
for file in os.listdir("reuters_data/"):
    
        filename = os.path.join("reuters_data", file)
        f = open(filename, 'r', encoding='utf-8', errors='ignore')
        dataFile = f.read()
        
        soup = BeautifulSoup(dataFile, 'html.parser')
        contents = soup.findAll('body')
        
        for content in contents:
            documents.append(content.text)

In [4]:
print('Number of documents: {}'.format(len(documents)))

Number of documents: 19043


In [5]:
print(documents[0])

Inco Ltd said it did not expect its
earlier reported removal from the Dow Jones industrial index to
make a major impact on the company's stock.
    "We don't think that individuals or institutions buy our
shares because we were one of the Dow Jones industrials,"
spokesman Ken Cherney said in reply to a query.
    Inco closed 1-3/8 lower at 19-3/8 in second most active
trading on the Toronto Stock Exchange.
    The Wall Street Journal, which selects the index, said Inco
was dropped to make the index more representative of the
market. Inco, the non-Communist world's largest nickel
producer, was a member of the index since 1928.
    Replacing Inco and Owens-Illinois Inc will be Coca-Cola Co
and Boeing Co, effective tomorrow.
    Nickel analyst Ilmar Martens at Walwyn Stodgell Cochran
Murray Ltd said Inco's removal from the index would likely
spark short-term selling pressure on the stock.
    "Some investors who have Inco may suddenly say, 'well,
because it's not now a Dow stock, we shoul

### Remove 'Reauter and ' from each document

In [6]:
new_doc = []
for d in documents:
    new_doc.append(d[:-8])
print(new_doc[0])

Inco Ltd said it did not expect its
earlier reported removal from the Dow Jones industrial index to
make a major impact on the company's stock.
    "We don't think that individuals or institutions buy our
shares because we were one of the Dow Jones industrials,"
spokesman Ken Cherney said in reply to a query.
    Inco closed 1-3/8 lower at 19-3/8 in second most active
trading on the Toronto Stock Exchange.
    The Wall Street Journal, which selects the index, said Inco
was dropped to make the index more representative of the
market. Inco, the non-Communist world's largest nickel
producer, was a member of the index since 1928.
    Replacing Inco and Owens-Illinois Inc will be Coca-Cola Co
and Boeing Co, effective tomorrow.
    Nickel analyst Ilmar Martens at Walwyn Stodgell Cochran
Murray Ltd said Inco's removal from the index would likely
spark short-term selling pressure on the stock.
    "Some investors who have Inco may suddenly say, 'well,
because it's not now a Dow stock, we shoul

### Join the documents

In [7]:
data = ""
for d in new_doc:
    data += d
len(data)

15775658

In [8]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
print('Data length:', len(data))
print(data[:100])

Data length: 15466792
Inco Ltd said it did not expect itsearlier reported removal from the Dow Jones industrial index toma


### Tokenization

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[6910,
 109,
 4,
 10,
 259,
 40,
 674,
 31970,
 265,
 4166,
 21,
 1,
 2017,
 2261,
 431,
 508,
 3265,
 6,
 192,
 846,
 12,
 1,
 294,
 66,
 95,
 948,
 638,
 17,
 3005,
 60,
 1071,
 288,
 31971,
 150,
 95,
 52,
 54,
 2,
 1,
 2017,
 2261,
 8654,
 156,
 14334,
 20399,
 4,
 5,
 3835,
 3,
 6,
 8308,
 6910,
 667,
 15,
 39,
 56,
 271,
 23,
 282,
 39,
 56,
 5,
 383,
 266,
 15848,
 12,
 1,
 1799,
 66,
 94,
 1,
 1214,
 1215,
 4772,
 42,
 31972,
 1,
 508,
 4,
 31973,
 1173,
 3,
 308,
 1,
 508,
 83,
 1594,
 2,
 1927,
 6910,
 1,
 272,
 2934,
 1392,
 530,
 31974,
 27,
 6,
 600,
 2,
 1,
 508,
 219,
 20400,
 3495,
 6910,
 7,
 6513,
 1871,
 50,
 18,
 22,
 4417,
 3836,
 10283,
 1667,
 75,
 545,
 847,
 4565,
 614,
 31975,
 9795,
 23,
 24629,
 20401,
 31976,
 109,
 4,
 31977,
 4166,
 21,
 1,
 508,
 37,
 31978,
 416,
 214,
 482,
 720,
 12,
 1,
 66,
 107,
 342,
 198,
 49,
 6910,
 79,
 7712,
 400,
 31979,
 150,
 826,
 40,
 160,
 6,
 2017,
 66,
 95,
 207,
 2898,
 31980,
 6685,
 4,
 9795,
 528,
 28,
 135,
 1,
 

In [10]:
unique_words = np.unique(encoded_data)
print('Unique words length: ', len(unique_words))
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

Unique words length:  133585


### Feature engineering

In [12]:
WORD_LENGTH = 5
prev_words = []
next_words = []
    
for i in range(len(encoded_data) - WORD_LENGTH):
    prev_words.append(encoded_data[i:i + WORD_LENGTH])
    next_words.append(encoded_data[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

[6910, 109, 4, 10, 259]
40


### Define X and Y

In [13]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [ ]:
Y = to_categorical(Y, num_classes=len(unique_words))

### Build the model

In [21]:
model = Sequential()
model.add(LSTM(64, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

### Traing the model

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=50, epochs=20, shuffle=True).history

Epoch 1/20
  154/47126 [..............................] - ETA: 9:15:48 - loss: 0.0000e+00 - accuracy: 0.0065